In [1]:
import pandas as pd
import os

# ==========================================
# [설정] 기존 매니페스트 경로
# ==========================================
BASE_DIR = r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\golden_dataset'
MANIFEST_PATH = os.path.join(BASE_DIR, 'det_train_manifest_aligned_folds.csv')
SAVE_PATH = os.path.join(BASE_DIR, 'det_train_manifest_balanced_1to1.csv')

def create_balanced_manifest():
    if not os.path.exists(MANIFEST_PATH):
        print("❌ 매니페스트 파일을 찾을 수 없습니다.")
        return

    df = pd.read_csv(MANIFEST_PATH)

    # 1. 골절(Positive)과 정상(Negative) 분리
    pos_df = df[df['fracture'] == 1].reset_index(drop=True)
    neg_df = df[df['fracture'] == 0].reset_index(drop=True)

    pos_count = len(pos_df)
    neg_count = len(neg_df)

    print(f"📊 원본 데이터 통계:")
    print(f"   - Fracture (Positive): {pos_count}")
    print(f"   - Normal   (Negative): {neg_count}")

    # 2. 정상 슬라이스를 골절 슬라이스 개수만큼 샘플링
    # 만약 정상 데이터가 골절보다 적을 일은 없겠지만, 안전하게 처리
    if neg_count > pos_count:
        neg_sampled_df = neg_df.sample(n=pos_count, random_state=42).reset_index(drop=True)
    else:
        neg_sampled_df = neg_df

    # 3. 두 데이터 합치기
    balanced_df = pd.concat([pos_df, neg_sampled_df], axis=0).sample(frac=1, random_state=42).reset_index(drop=True)

    # 4. Fold 다시 나누기 (데이터가 줄어들었으므로 새로 나누는 것이 깔끔함)
    from sklearn.model_selection import KFold
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    balanced_df['fold'] = -1
    uids = balanced_df['StudyInstanceUID'].unique()
    
    for f_idx, (_, val_idx) in enumerate(kf.split(uids), 1):
        val_pids = uids[val_idx]
        balanced_df.loc[balanced_df['StudyInstanceUID'].isin(val_pids), 'fold'] = f_idx

    # 5. 저장
    balanced_df.to_csv(SAVE_PATH, index=False)
    
    print(f"\n✅ 1:1 균형 매니페스트 생성 완료!")
    print(f"   - 총 슬라이스 수: {len(balanced_df)}")
    print(f"   - 저장 경로: {SAVE_PATH}")

if __name__ == '__main__':
    create_balanced_manifest()

📊 원본 데이터 통계:
   - Fracture (Positive): 7205
   - Normal   (Negative): 45019

✅ 1:1 균형 매니페스트 생성 완료!
   - 총 슬라이스 수: 14410
   - 저장 경로: C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\golden_dataset\det_train_manifest_balanced_1to1.csv


## 학습

In [4]:
import os
# --- [1. OpenMP 에러 방지] ---
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import ast
import yaml
from pathlib import Path

# Ultralytics 공식 트레이너 관련
from ultralytics import YOLO
from ultralytics.models.yolo.detect import DetectionTrainer

# ==========================================
# 2. 설정 (Configuration)
# ==========================================
CONFIG = {
    'base_dir': r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\golden_dataset',
    
    # [핵심 변경] 방금 만든 1:1 균형 데이터셋 경로 연결
    'manifest_path': r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\golden_dataset\det_train_manifest_balanced_1to1.csv',
    
    'pretrained_model': 'yolo26s.pt', 
    'save_dir_root': './runs/rsna_5fold_balanced_1to1', # 저장 경로 구분
    'img_size': 512,
    'batch_size': 16,
    'epochs': 100,        # 1:1 비율이므로 50에폭 추천 (금방 돕니다)
    'patience': 30,
    'workers': 0,
    'folds_to_run': [1]
}

def create_dummy_structure(base_dir):
    fake_dirs = [
        os.path.join(base_dir, 'images', 'train'), os.path.join(base_dir, 'images', 'val'),
        os.path.join(base_dir, 'labels', 'train'), os.path.join(base_dir, 'labels', 'val')
    ]
    for d in fake_dirs: os.makedirs(d, exist_ok=True)

# ==========================================
# 3. 데이터셋 (512px 대응)
# ==========================================
class RSNA4ChannelDataset(Dataset):
    def __init__(self, df, base_dir):
        self.df = df
        self.base_dir = base_dir
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        full_path = os.path.join(self.base_dir, row['file_path'])
        
        try:
            with np.load(full_path) as loaded:
                img = loaded['data'].astype(np.float32)
                h, w = img.shape[1], img.shape[2] 
        except:
            img = np.zeros((4, CONFIG['img_size'], CONFIG['img_size']), dtype=np.float32)
            h, w = CONFIG['img_size'], CONFIG['img_size']

        boxes = [] 
        try:
            fracture_flag = int(float(row['fracture']))
        except:
            fracture_flag = 0
            
        if fracture_flag == 1 and pd.notna(row['bbox']):
            try:
                raw_boxes = row['bbox']
                if isinstance(raw_boxes, str):
                    if 'np.float64' in raw_boxes:
                        raw_boxes = raw_boxes.replace('np.float64(', '').replace(')', '')
                    raw_boxes = ast.literal_eval(raw_boxes)
                
                if isinstance(raw_boxes, list):
                    for box in raw_boxes:
                        if len(box) == 4:
                            x, y, w_box, h_box = box
                            cx, cy = x + w_box/2, y + h_box/2
                            try:
                                cls = int(float(row['ver_level'])) - 1 
                            except:
                                cls = -1
                            if 0 <= cls < 7:
                                boxes.append([cls, cx, cy, w_box, h_box])
            except Exception as e:
                pass
        
        boxes = torch.tensor(boxes, dtype=torch.float32) if boxes else torch.zeros((0, 5))
        img_tensor = torch.from_numpy(img)
        
        return img_tensor, boxes, (h, w), full_path

# [학습용] Ultralytics 포맷 Collate
def trainer_collate_fn(batch):
    images, box_lists, shapes, paths = zip(*batch)
    images = torch.stack(images, 0)
    
    new_boxes = []
    for i, boxes in enumerate(box_lists):
        if boxes.shape[0] > 0:
            idx_col = torch.full((boxes.shape[0], 1), i, dtype=torch.float32)
            combined = torch.cat([idx_col, boxes], dim=1)
            new_boxes.append(combined)
    targets = torch.cat(new_boxes, 0) if new_boxes else torch.zeros((0, 6))
    
    return {
        'img': images,
        'batch_idx': targets[:, 0],
        'cls': targets[:, 1].view(-1, 1),
        'bboxes': targets[:, 2:],
        'device': None,
        'ori_shape': list(shapes),
        'resized_shape': list(shapes),
        'im_file': list(paths),
        'ratio_pad': [((1.0, 1.0), (0.0, 0.0)) for _ in range(len(images))] 
    }

# [검증용] 단순 Collate (박스 합치지 않음 -> 에러 해결!)
def val_collate_fn(batch):
    images, box_lists, shapes, paths = zip(*batch)
    # 이미지만 쌓고, 박스는 리스트(튜플) 상태로 유지
    images = torch.stack(images, 0)
    return images, box_lists, shapes, paths

# ==========================================
# 4. 커스텀 트레이너
# ==========================================
class RSNATrainer(DetectionTrainer):
    def __init__(self, overrides=None, current_fold=1):
        self.current_fold = current_fold
        super().__init__(overrides=overrides)

    def build_dataset(self, img_path, mode="train", batch=None):
        df = pd.read_csv(CONFIG['manifest_path'])
        if mode == "train":
            sub_df = df[df['fold'] != self.current_fold].reset_index(drop=True)
        else:
            sub_df = df[df['fold'] == self.current_fold].reset_index(drop=True)
            
        print(f"Dataset Build ({mode}): {len(sub_df)} samples loaded.")
        return RSNA4ChannelDataset(sub_df, CONFIG['base_dir'])

    def get_dataloader(self, dataset_path, batch_size=16, rank=0, mode="train"):
        if hasattr(self, 'data') and isinstance(self.data, dict):
            self.data['channels'] = 4
            self.data['ch'] = 4
            
        dataset = self.build_dataset(dataset_path, mode)
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=(mode == "train"),
                            num_workers=0, collate_fn=trainer_collate_fn, pin_memory=True)
        loader.reset = lambda: None
        return loader

    def get_model(self, cfg=None, weights=None, verbose=True):
        target_weights = weights if weights else 'yolov8s.pt'
        print(f"📦 [Trainer] 모델 로드 시도: {target_weights}")
        try:
            temp_yolo = YOLO(target_weights)
        except Exception as e:
            print(f"⚠️ {target_weights} 로드 실패, yolov8s.pt로 대체합니다.")
            target_weights = 'yolov8s.pt'
            temp_yolo = YOLO(target_weights)

        model = temp_yolo.model
        m = model.model[0]
        if isinstance(m.conv, nn.Conv2d) and m.conv.in_channels == 3:
            old_conv = m.conv
            new_conv = nn.Conv2d(4, old_conv.out_channels, 
                                 kernel_size=old_conv.kernel_size, stride=old_conv.stride, 
                                 padding=old_conv.padding, bias=(old_conv.bias is not None))
            with torch.no_grad():
                new_conv.weight[:, :3] = old_conv.weight
                new_conv.weight[:, 3] = old_conv.weight.mean(dim=1)
                if old_conv.bias is not None: new_conv.bias = old_conv.bias
            m.conv = new_conv
            m.c1 = 4
            print("🔧 Conv1 4채널 확장 완료")
        return model

    def final_eval(self):
        print("⚠️ [Info] Ultralytics 내부 final_eval 건너뜀 (4채널 호환 문제 방지)")
        return

    def plot_training_labels(self, *args, **kwargs): pass 

# ==========================================
# 5. 커스텀 메트릭 (수정됨)
# ==========================================
def calculate_custom_metrics(model_path, val_fold, device):
    print(f"\n📊 [Fold {val_fold}] 커스텀 메트릭 계산 시작...")
    
    df = pd.read_csv(CONFIG['manifest_path'])
    val_df = df[df['fold'] == val_fold].reset_index(drop=True)
    val_ds = RSNA4ChannelDataset(val_df, CONFIG['base_dir'])
    
    # [핵심] collate_fn=val_collate_fn 사용 (박스 Stack 에러 방지)
    val_loader = DataLoader(val_ds, batch_size=CONFIG['batch_size'], shuffle=False, 
                            num_workers=0, collate_fn=val_collate_fn)

    temp_trainer = RSNATrainer(overrides={'model': 'yolov8s.pt', 'data': 'rsna_dummy.yaml'}, current_fold=val_fold)
    
    if os.path.exists(model_path):
        model = temp_trainer.get_model(weights=str(model_path))
    else:
        print(f"❌ [Critical] 가중치 파일 없음: {model_path}")
        return 0.0, 0.0

    model.to(device)
    model.eval()

    preds_prob = []
    targets = []

    with torch.no_grad():
        for img, boxes, _, _ in val_loader:
            img = img.to(device)
            # 순수 추론 (4채널)
            output = model(img) 
            
            # YOLOv8 Output: (Batch, 4+Cls, Anchors)
            # 보통 output[0]이 추론 결과임
            if isinstance(output, (list, tuple)):
                output = output[0]
                
            pred_tensor = output.transpose(1, 2) # (Batch, Anchors, 4+Cls)
            
            for i in range(len(img)):
                p = pred_tensor[i]
                # 4번째 인덱스부터 클래스 확률
                cls_probs = p[:, 4:] 
                conf, _ = cls_probs.max(1) # 가장 높은 확률 하나 (Anchor 중)
                preds_prob.append(conf.max().item()) # 이미지 전체에서 가장 높은 확률
            
            # 박스(Target) 처리
            for b in boxes:
                targets.append(1.0 if b.shape[0] > 0 else 0.0)

    preds_prob = np.array(preds_prob)
    targets = np.array(targets)
    
    # Accuracy & LogLoss 계산
    acc = ((preds_prob > 0.5) == targets).mean()
    epsilon = 1e-15
    preds_prob = np.clip(preds_prob, epsilon, 1 - epsilon)
    log_loss = -(targets * np.log(preds_prob) + (1 - targets) * np.log(1 - preds_prob)).mean()

    print(f"   👉 Accuracy: {acc:.4f}")
    print(f"   👉 Log Loss: {log_loss:.4f}")
    return acc, log_loss

# ==========================================
# 6. 메인 실행
# ==========================================
if __name__ == '__main__':
    # 1. 저장 폴더 미리 생성
    save_root_abs = os.path.abspath(CONFIG['save_dir_root'])
    os.makedirs(save_root_abs, exist_ok=True)
    
    create_dummy_structure(CONFIG['base_dir'])
    
    # 2. YAML 생성 (절대 경로)
    current_dir = os.getcwd()
    dummy_yaml_path = os.path.abspath(os.path.join(current_dir, "rsna_dummy.yaml"))
    
    with open(dummy_yaml_path, 'w') as f:
        yaml.dump({
            'path': os.path.abspath(CONFIG['base_dir']), 
            'train': 'images/train', 
            'val': 'images/val',
            'nc': 7, 
            'names': ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'],
            'ch': 4 
        }, f)

    final_results = []
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # 3. 매니페스트 파일 확인
    if not os.path.exists(CONFIG['manifest_path']):
        print(f"❌ [Error] 1:1 매니페스트 파일이 없습니다: {CONFIG['manifest_path']}")
        exit()

    print(f"🔥 학습 시작! (1:1 Balanced Data)")
    print(f"📂 데이터셋: {CONFIG['manifest_path']}")

    for fold in CONFIG['folds_to_run']:
        print(f"\n{'='*40}")
        print(f"🚀 FOLD {fold} 학습 시작")
        print(f"{'='*40}")
        
        project_path = os.path.abspath(CONFIG['save_dir_root'])
        name_path = f'fold_{fold}'
        
        args = dict(
            model=CONFIG['pretrained_model'], 
            data=dummy_yaml_path, 
            epochs=100,             # 넉넉하게 100
            batch=CONFIG['batch_size'], 
            imgsz=CONFIG['img_size'], 
            device=0 if device == 'cuda' else 'cpu',
            project=project_path,
            name=name_path, 
            exist_ok=True, 
            workers=0,
            val=True, 
            plots=True,             # 학습 과정을 그림으로 확인하기 위해 True
            amp=True,               # 가속화 유지
            save=True,
    
            # --- [의료 영상 최적화 핵심 설정] ---
            patience=30,            # 너무 빨리 끄지 않도록 인내심 상향
            mosaic=0.0,             # [중요] 의료 영상에서 모자이크는 독입니다. 끄세요.
            mixup=0.0,              # 믹스업도 끕니다.
            copy_paste=0.0,         # 카피페이스트도 끕니다.
            degrees=5.0,            # 아주 미세한 회전만 허용
            flipud=0.0,             # 척추가 뒤집힐 일은 없으니 상하반전 끕니다.
            fliplr=0.5,             # 좌우 반전은 해부학적으로 가능하니 유지
    
            # --- [손실 함수 튜닝] ---
            box=10.0,               # [중요] 박스를 맞췄을 때 상점을 7.5 -> 10.0으로 상향
            cls=1.0,                # 클래스 분류 가중치 유지
            warmup_epochs=5.0,      # 초반에 천천히 배울 수 있도록 예열 기간 3 -> 5
            lr0=0.005,              # 초기 학습률을 살짝 낮춰서 신중하게 시작
        )

        trainer = RSNATrainer(overrides=args, current_fold=fold)
        trainer.train()
        
        # 실제 저장 경로 찾기
        real_save_dir = trainer.save_dir 
        best_pt = real_save_dir / 'weights' / 'best.pt'
        if not best_pt.exists():
            best_pt = real_save_dir / 'weights' / 'last.pt'
            
        # 커스텀 메트릭 계산
        acc, log_loss = calculate_custom_metrics(best_pt, fold, device)
        
        # YOLO 자체 결과(mAP) 가져오기
        yolo_res_path = real_save_dir / 'results.csv'
        map50 = 0.0
        if yolo_res_path.exists():
            try:
                res_df = pd.read_csv(yolo_res_path)
                res_df.columns = [c.strip() for c in res_df.columns]
                map50 = res_df['metrics/mAP50(B)'].iloc[-1]
            except: pass

        final_results.append({'fold': fold, 'accuracy': acc, 'log_loss': log_loss, 'mAP50': map50})
        
        # 중간 저장
        pd.DataFrame(final_results).to_csv(os.path.join(save_root_abs, 'rsna_final_summary.csv'), index=False)

    print(f"\n✨ 모든 Fold 학습 완료!")
    print(pd.DataFrame(final_results))

🔥 학습 시작! (1:1 Balanced Data)
📂 데이터셋: C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\golden_dataset\det_train_manifest_balanced_1to1.csv

🚀 FOLD 1 학습 시작
Ultralytics 8.4.14  Python-3.12.12 torch-2.9.1+cu130 CUDA:0 (NVIDIA GeForce RTX 5060 Ti, 16311MiB)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=10.0, cache=False, cfg=None, classes=None, close_mosaic=10, cls=1.0, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=c:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\golden_dataset\rsna_dummy.yaml, degrees=5.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=

KeyboardInterrupt: 